In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel("../data/multi_asset_etf_data.xlsx", sheet_name="excess returns")
df.set_index("Date", inplace=True)
df.head()

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2009-04-30,0.008441,-0.001553,0.155030,0.114637,0.137907,-0.028004,0.295598,0.229650,0.022329,0.098793,-0.018505
2009-05-31,0.054143,0.163134,0.159871,0.132389,0.029026,-0.020303,0.023198,0.054363,0.028336,0.058925,0.020437
2009-06-30,0.004550,-0.026858,-0.023094,-0.014648,0.032919,-0.006170,-0.025462,0.041443,-0.004035,-0.001254,0.001383
2009-07-31,0.031312,0.018595,0.110173,0.100442,0.069217,0.008344,0.105826,0.143274,0.015353,0.074633,0.000906
2009-08-31,0.007193,-0.040800,-0.013571,0.044595,-0.017404,0.007199,0.131504,0.032977,-0.004586,0.036504,0.007979


# 1. HMC's Approach

### 1. 
There are thousands of individual risky assets in which HMC can invest.  Explain why MV optimization across 1,000 securities is infeasible.

Answer: 
Because calculate covariance matrix of 1000 securities is a resource expensive process.

### 2.
Rather than optimize across all securities directly, HMC runs a two-stage optimization.
1. They build asset class portfolios with each one optimized over the securities of the specific asset class.  
2. HMC combines the asset-class portfolios into one total optimized portfolio.

In order for the two-stage optimization to be a good approximation of the full MV-optimization on all assets, what must be true of the partition of securities into asset classes?

Answer: 
Covariance within asset classes must be much higher than across asset classes. 
??????

### 3.
Should TIPS form a new asset class or be grouped into one of the other 11 classes?

Answer: 
Separate. See Q2.4. 

### 4. 
Why does HMC focus on real returns when analyzing its portfolio allocation? Is this just a matter of scaling, or does using real returns versus nominal returns potentially change the MV solution?

Answer:
Because the targets of HMC is to contribute 4 ~ 5% to different schools for their spending while preserve the endowment balance, if using nominal returns, there would be a mismatch between this targets and solution due to inflation.
?????

### 5.
The case discusses the fact that Harvard places bounds on the portfolio allocation rather than implementing whatever numbers come out of the MV optimization problem.

How might we adjust the stated optimization problem in the lecture notes to reflect the extra constraints Harvard is using in their bounded solutions given in Exhibits 5 and 6. Just consider how we might rewrite the optimization; don’t try to solve this extra-constrained optimization.

Answer:
Exhibit 5:
1. weight-i in [0, 1] for i in each non-cash asset class
2. Cash's constraint is [-0.5, 1]

Exhibit 6:
1. weight-tips in [0, 1]
1. weight-i in [weight-i-1999 - 0.1, weight-i-1999 + 0.1] for i in each non-cash asset class
(Cash's constraint is [-0.5, 1] but since 1999 cash weight = -0.05, and by constraints (2), no need to add extra constraint for cash)

### 6. 
Exhibits 5 shows zero allocation to domestic equities and domestic bonds across the entire computed range of targeted returns, (5.75% to 7.25%). Conceptually, why is the constraint binding in all these cases? What would the unconstrained portfolio want to do with those allocations and why?

Answer:
Domestic Equity: h

### 7.
Exhibit 6 changes the constraints, (tightening them in most cases.) How much deterioration do we see in the mean-variance tradeoff that Harvard achieved?

# 2. Mean-Variance Optimization

In [4]:
## 1. Summary Statistics
def performance_stat(s: pd.Series) -> pd.Series:
    """
    Calculate the mean, volatility, sharpe of given series
    
    Parameters:
        s (pd.Series): 
            Excess return of certain asset / portfolio
            Index: all time period (i.e. monthly)
            
    Returns:
        s_stat: Series contains mean, volatility, sharpe of the input series
    """
    s_stat = s.agg(['mean', 'std']).T
    s_stat['mean'] *= 12
    s_stat['std'] *= (12 ** (1/2))
    s_stat['sharpe'] = s_stat['mean'] / s_stat['std']
    return s_stat

In [8]:
df_stat = df.apply(performance_stat).T
df_stat.sort_values('sharpe', inplace=True)
print(f"Best performer: {df_stat.index[-1]}; Worse performer: {df_stat.index[0]}")
df_stat

Best performer: SPY; Worse performer: BWX


,mean,std,sharpe
BWX,-0.001843,0.083359,-0.022112
DBC,0.025443,0.178975,0.142162
IEF,0.014269,0.062405,0.228652
EEM,0.064887,0.196531,0.330163
PSP,0.079938,0.227387,0.351552
QAI,0.018974,0.050810,0.373440
TIP,0.022321,0.051529,0.433166
EFA,0.081597,0.165991,0.491573
IYR,0.129473,0.187101,0.691997
HYG,0.064168,0.089154,0.719746


In [10]:
## 2. Descriptive Analysis
df_corr = df.corr()
df_corr_unstack = df_corr.unstack().sort_values()
df_corr_unstack = df_corr_unstack[df_corr_unstack < 1]
print(f"Highest correlation: {df_corr_unstack.index[-1]}; Lowest correlation: {df_corr_unstack.index[0]}")
df_corr

Highest correlation: ('PSP', 'SPY'); Lowest correlation: ('DBC', 'IEF')


,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
BWX,1.000000,0.349773,0.647614,0.621662,0.557653,0.434472,0.453534,0.524870,0.668045,0.465713,0.617099
DBC,0.349773,1.000000,0.565654,0.581865,0.473208,-0.321738,0.318314,0.496057,0.547936,0.509886,0.136668
EEM,0.647614,0.565654,1.000000,0.851579,0.726041,-0.102347,0.621814,0.771677,0.807245,0.734556,0.302729
EFA,0.621662,0.581865,0.851579,1.000000,0.771463,-0.132331,0.697875,0.891929,0.853674,0.871641,0.287476
HYG,0.557653,0.473208,0.726041,0.771463,1.000000,-0.008598,0.757649,0.823823,0.768756,0.770353,0.365939
IEF,0.434472,-0.321738,-0.102347,-0.132331,-0.008598,1.000000,0.073622,-0.118676,0.055667,-0.155696,0.706078
IYR,0.453534,0.318314,0.621814,0.697875,0.757649,0.073622,1.000000,0.760158,0.655963,0.753610,0.397166
PSP,0.524870,0.496057,0.771677,0.891929,0.823823,-0.118676,0.760158,1.000000,0.838287,0.895729,0.320913
QAI,0.668045,0.547936,0.807245,0.853674,0.768756,0.055667,0.655963,0.838287,1.000000,0.840989,0.459712
SPY,0.465713,0.509886,0.734556,0.871641,0.770353,-0.155696,0.753610,0.895729,0.840989,1.000000,0.294639


In [92]:
print("By table 1, TIP has higher mean return and sharpe ratio than IEF (domestic bonds) and BWX (foreign bonds).")

By table 1, TIP has higher mean return and sharpe ratio than IEF (domestic bonds) and BWX (foreign bonds).


In [5]:
## 3. The MV frontier
def mv_frontier_tang(df: pd.DataFrame, is_reg: bool = False) -> pd.Series:
    """
    Calculate the tangent portfolio weight on efficient frontier
    
    Parameters:
        df (pd.DataFrame): 
            Excess return df.
            Columns: all assets
            Index: all time period (i.e. monthly)
        is_reg (bool): 
            If True, use regularized covariance matrix.
            
    Returns:
        weight_tang: tangent portfolio weight in ascending order
    """
    df_cov_ann = df.cov()
    if is_reg:
        df_cov_ann += np.diag(np.diag(df_cov_ann))
        df_cov_ann /= 2
    df_cov_mean = pd.Series(np.linalg.inv(df_cov_ann) @ df.mean(), 
                            index=df_cov_ann.columns.to_list(),
                            name='Tangency Weight')
    df_tang = df_cov_mean / df_cov_mean.sum()
    df_tang.sort_values(inplace=True)
    return df_tang

In [15]:
weight_tang = mv_frontier_tang(df)
weight_tang

QAI   -3.133445
BWX   -1.464974
PSP   -1.271055
IYR   -0.242772
DBC    0.028436
EEM    0.261028
TIP    0.356935
EFA    0.452914
HYG    1.528942
IEF    1.893992
SPY    2.589999
Name: Tangency Weight, dtype: float64

In [74]:
print(f"The ranking of weight of tangency portfolio "
      f"{'=' if all(df_stat.index == weight_tang.index) else '!='} sharpe ratio rank.")

The ranking of weight of tangency portfolio != sharpe ratio rank.


In [75]:
df_tang = (df * weight_tang.loc[df.columns]).sum(axis=1)
df_tang_stat = performance_stat(df_tang)
df_tang_stat

mean      0.370180
std       0.191523
sharpe    1.932824
dtype: float64

In [76]:
## 4. TIPS

# a) without TIPS
weight_tang_excl_tips = mv_frontier_tang(df.drop(columns=["TIP"]))
df_tang_excl_tips = (df * weight_tang_excl_tips.loc[df.drop(columns=["TIP"]).columns]).sum(axis=1)
performance_stat(df_tang_excl_tips)

mean      0.386291
std       0.200111
sharpe    1.930383
dtype: float64

In [77]:
# Change of tangent portfolio
weight_tang_excl_tips - weight_tang

BWX   -0.047776
DBC    0.026721
EEM    0.017058
EFA   -0.011418
HYG    0.064197
IEF    0.318459
IYR   -0.003123
PSP   -0.043037
QAI   -0.105510
SPY    0.141364
TIP         NaN
Name: Tangency Weight, dtype: float64

In [78]:
# a) TIPS += 0.0012
df_tip_adj = df.copy()
df_tip_adj["TIP"] += 0.0012
weight_tang_adj_tips = mv_frontier_tang(df_tip_adj)
df_tang_adj_tips = (df_tip_adj * weight_tang_adj_tips.loc[df_tip_adj.columns]).sum(axis=1)
performance_stat(df_tang_adj_tips)

mean      0.328908
std       0.161947
sharpe    2.030967
dtype: float64

In [79]:
# Change of tangent portfolio
weight_tang_adj_tips - weight_tang

BWX    0.202111
DBC   -0.113042
EEM   -0.072163
EFA    0.048303
HYG   -0.271582
IEF   -1.347217
IYR    0.013213
PSP    0.182065
QAI    0.446354
SPY   -0.598029
TIP    1.509987
Name: Tangency Weight, dtype: float64

In [80]:
# Based on the results, TIPS should be not considered as a separate assets
# 1. sharpe ratio of tangent portfolio with/without TIPS is almost the same (with-TIPS has marginal advantage);
# 2. when TIPS increase overall, tangent portfolio has a higher sharpe

# 3. Allocation

In [15]:
def calculate_weight(df_train: pd.DataFrame, method: str == "tang") -> pd.Series:
    if method == "ew":
        return pd.Series([1/df_train.shape[1]] * df_train.shape[1], index=df_train.columns)
    elif method == "rp":
        rp_weight = 12 / df_train.var()
        rp_weight = rp_weight / rp_weight.sum()
        return rp_weight
    elif method == "reg":
        return mv_frontier_tang(df_train, is_reg=True)
    elif method == "tang":
        return mv_frontier_tang(df_train, is_reg=False)

def construct_portfolio(df_test: pd.DataFrame, weight: pd.Series) -> pd.Series:
    port = (df_test * weight.filter(df_test.columns)).sum(axis=1)
    port *= 0.01 / port.mean()  # rescale for target return (i.e. weights not sum to 1)
    return port

In [16]:
weight_ew = calculate_weight(df, method="ew")
df_ew = construct_portfolio(df, weight_ew)    
df_ew

Date
2009-04-30    0.189311
2009-05-31    0.131303
2009-06-30   -0.003961
2009-07-31    0.126554
2009-08-31    0.035758
                ...   
2023-04-30    0.009788
2023-05-31   -0.054093
2023-06-30    0.049935
2023-07-31    0.052559
2023-08-31   -0.049650
Length: 173, dtype: float64

In [17]:
weight_rp = calculate_weight(df, method="rp")
df_rp = construct_portfolio(df, weight_rp)    
df_rp

Date
2009-04-30    0.104082
2009-05-31    0.116919
2009-06-30    0.000589
2009-07-31    0.102312
2009-08-31    0.021531
                ...   
2023-04-30    0.005043
2023-05-31   -0.070606
2023-06-30    0.026585
2023-07-31    0.031494
2023-08-31   -0.058573
Length: 173, dtype: float64

In [18]:
weight_reg = calculate_weight(df, method="reg")
df_reg = construct_portfolio(df, weight_reg)    
df_reg

Date
2009-04-30    0.104374
2009-05-31    0.010093
2009-06-30    0.002777
2009-07-31    0.056504
2009-08-31    0.033247
                ...   
2023-04-30    0.009056
2023-05-31   -0.007136
2023-06-30    0.024594
2023-07-31    0.001026
2023-08-31   -0.007315
Length: 173, dtype: float64

In [21]:
# df_tang_rescale = df_tang * 0.01 / df_tang.mean() 

stat_comp = pd.concat([
    # performance_stat(df_tang_rescale).rename('Tangent'), 
    performance_stat(df_ew).rename('Equal Weight'),
    performance_stat(df_rp).rename('Risk Parity'),
    performance_stat(df_reg).rename('Regularized')
], axis=1)
stat_comp

,Equal Weight,Risk Parity,Regularized
mean,0.120000,0.120000,0.120000
std,0.213275,0.217685,0.094096
sharpe,0.562655,0.551256,1.275299


# 4. Out-of-Sample Performance

In [51]:
from functools import partial 
is_oos = df.index >= pd.to_datetime('2021-01-01')
df_train = df.loc[~is_oos]

test_stats = []
train_stats = []
for method in ["ew", "rp", "reg", "tang"]:
    weight_ = calculate_weight(df_train, method=method)
    for results, df_test in [(train_stats, df.loc[~is_oos]), 
                             (test_stats, df.loc[is_oos])]:
        stat_ =  (df_test
                  .pipe(partial(construct_portfolio, weight=weight_))
                  .pipe(performance_stat))
        results.append(stat_.rename(method).copy())

In [52]:
# train sample stats
pd.concat(train_stats, axis=1).T.sort_values("sharpe", ascending=False)

,mean,std,sharpe
tang,0.12,0.047744,2.513379
reg,0.12,0.072802,1.648316
rp,0.12,0.124563,0.963370
ew,0.12,0.164403,0.729912


In [53]:
# test sample stats
pd.concat(test_stats, axis=1).T.sort_values("sharpe", ascending=False)

,mean,std,sharpe
rp,0.12,0.222654,0.538954
reg,0.12,0.351896,0.341009
tang,0.12,0.593915,0.202049
ew,0.12,1.810709,0.066272


In [56]:
# 4.2 
ann_test_stats = []

for cut_off in range(2014, 2023):
    is_oos = df.index >= pd.to_datetime(f'{cut_off}-01-01')
    df_train = df.loc[df.index < pd.to_datetime(f'{cut_off}-01-01')]
    df_test = df.loc[(df.index >= pd.to_datetime(f'{cut_off}-01-01')) & 
                     (df.index < pd.to_datetime(f'{cut_off + 1}-01-01'))]
    for method in ["ew", "rp", "reg", "tang"]:
        weight_ = calculate_weight(df_train, method=method)
        stat_ = (df.loc[is_oos]
                .pipe(partial(construct_portfolio, weight=weight_))
                .pipe(performance_stat))
        ann_test_stats.append(stat_.rename(f"{method}-{cut_off}").copy())

In [57]:
pd.concat(ann_test_stats, axis=1).T.sort_values("sharpe", ascending=False)

,mean,std,sharpe
tang-2014,0.12,0.097059,1.236356
tang-2015,0.12,0.109798,1.092911
tang-2016,0.12,0.113510,1.057177
tang-2017,0.12,0.119583,1.003486
rp-2022,0.12,0.133026,0.902081
tang-2018,0.12,0.133989,0.895593
tang-2019,0.12,0.135987,0.882441
reg-2022,0.12,0.166296,0.721603
tang-2020,0.12,0.182055,0.659141
reg-2014,0.12,0.199035,0.602909
